# CMPINF-0010 Final Project (The Foodies)

## My Metric

Like previously stated, our group has determined that the best neighborhood in Pittsburgh is the one that has the best food. My dataset contains all of the food gardens in Pittsburgh, and I am going to determine how many food gardens are in each Pittsburgh neighborhood. More food gardens correlates to better neighborhood, and I will begin my exploration by loading in the dataset and viewing a few of the rows. 

In [1]:
import pandas as pd 
import geopandas as gpd

data = pd.read_csv('data/GrowGardenData.csv')
data.head(4)

,urban_grower,grower_id,category,url,street_address,city,state,zip_code,country,latitude,longitude
0,Angora Gardens,584,sustainability-fund-site|other,https://www.growpittsburgh.org/directory/garde...,3 Muse Lane,White Oak,Pennsylvania,15131.0,US,40.404686,-79.862095
1,Everybody's Garden,586,sustainability-fund-site|other,https://www.growpittsburgh.org/directory/garde...,301 West Elizabeth Street,Pittsburgh,Pennsylvania,15207.0,US,40.406546,-79.945442
2,EvGard Extension,587,other,https://www.growpittsburgh.org/directory/garde...,NaN,Pittsburgh,Pennsylvania,15201.0,US,40.406377,-79.945559
3,Glen Hazel Communiuty Garden,588,other,https://www.growpittsburgh.org/directory/garde...,Johnston Avenue at Roselle Drive,Pittsburgh,Pennsylvania,15207.0,US,40.406040,-79.931253


For each grow garden in Pittsburgh, this dataset contains it's name, a unique ID, the type of garden, it's city, state, zip code, country, latitude, and longitude. 

I am going to drop a couple of columns from the dataset which are not useful in determining which gardens are in which neighborhood, in order to tidy up the dataset. 

In [2]:
data1 = data.drop(columns=["category","url","state","country"])
data1.head(4)

,urban_grower,grower_id,street_address,city,zip_code,latitude,longitude
0,Angora Gardens,584,3 Muse Lane,White Oak,15131.0,40.404686,-79.862095
1,Everybody's Garden,586,301 West Elizabeth Street,Pittsburgh,15207.0,40.406546,-79.945442
2,EvGard Extension,587,NaN,Pittsburgh,15201.0,40.406377,-79.945559
3,Glen Hazel Communiuty Garden,588,Johnston Avenue at Roselle Drive,Pittsburgh,15207.0,40.406040,-79.931253


The columns I have removed are category, url, state and country. The category and url of each grow garden is not important in determining where the gardens are located. Also, all of the gardens are located in Pennsylvania and the US, so the state and country columns are the exact same for each garden. 

Next, I am going to load in the data of Pittsburgh neighborhoods and determine how many gardens are in each neighborhood. 